In [1]:
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm
from gensim.models import KeyedVectors
import gc

In [2]:
train_path = '../../user_data/dataset/'
test_path = '../../user_data/dataset/'
header = 'underexpose'
now_phase = 9

In [3]:
file_name = 'recall_0531_addsim_addAA_RA_additemtime_addcount_addnn_addtxt'
df = pd.read_csv(train_path + 'new_recall/' + file_name + '.csv')

In [4]:
click_trn = []
click_tst = []
#qtime_tst = []
for p in tqdm(range(0, now_phase+1)):
    tmp = pd.read_csv(train_path + header + f'_train_click-{p}.csv', header=None, names=['user_id', 'item_id', 'time'])
    tmp['phrase'] = p
    click_trn.append(tmp)
    tmp = pd.read_csv(test_path + header + f'_test_click-{p}.csv', header=None, names=['user_id', 'item_id', 'time'])
    tmp['phrase'] = p
    click_tst.append(tmp)
    #tmp = pd.read_csv(test_path + header + f'_test_qtime-{p}.csv', header=None, names=['user_id', 'item_id', 'query_time'])
    #tmp['phrase'] = p
    #qtime_tst.append(tmp)
    
click_trn = pd.concat(click_trn, axis=0, ignore_index=True)
click_tst = pd.concat(click_tst, axis=0, ignore_index=True)
#qtime_tst = pd.concat(qtime_tst, axis=0, ignore_index=True)

100%|██████████| 10/10 [00:00<00:00, 10.52it/s]


In [5]:
click_df = pd.concat([click_trn, click_tst], axis=0, ignore_index=True)
click_df['item_count'] = click_df.groupby(['item_id', 'phrase'])['user_id'].transform('count')

In [6]:
click_df.shape

(2910846, 5)

In [7]:
click_df.head()

user_id  item_id      time  phrase  item_count
0     4965       18  0.983763       0           4
1    20192       34  0.983772       0          12
2    30128       91  0.983780       0           9
3    29473      189  0.983930       0           4
4    10625      225  0.983925       0           6

In [8]:
count_map = click_df[['item_id', 'phrase', 'item_count']].drop_duplicates()

In [9]:
df.columns

Index(['user_id', 'item_id', 'sim', 'feature_0', 'feature_1', 'feature_2',
       'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7',
       'feature_8', 'feature_9', 'feature_10', 'cn_sim', 'hpi_sim', 'hdi_sim',
       'lhn1_sim', 'ra_sim', 'aa_sim', 'time_interval', 'item_count_12h',
       'item_count_6h', 'item_count_4h', 'item_count_2h', 'item_count_1h',
       'phrase', 'count', 'diff_from_last', 'diff_from_next', 'diff_from_max',
       'nn', 'txt_feature_0', 'txt_feature_1', 'txt_feature_2',
       'txt_feature_3', 'txt_feature_4', 'img_feature_0', 'img_feature_1',
       'img_feature_2', 'img_feature_3', 'img_feature_4'],
      dtype='object')

In [10]:
def gen_add_1(df):
    group_df = df.groupby(['user_id', 'phrase', 'item_count'])['time'].agg([['user_item_count_cnt', 'count'],
                                                                            ['user_item_count_max_time', 'max'],
                                                                            ['user_item_count_min_time', 'min']]).reset_index()
    group_df['sum'] = group_df.groupby(['user_id', 'phrase'])['user_item_count_cnt'].transform('sum')
    group_df['user_item_count_ratio'] = group_df['user_item_count_cnt'] / group_df['sum']
    group_df['user_item_count_timedelta'] = group_df['user_item_count_max_time'] - group_df['user_item_count_min_time']
    del group_df['sum']
    return group_df

In [11]:
df = df.merge(count_map, on=['item_id', 'phrase'], how='left')

In [12]:
df.shape

(17383059, 43)

In [13]:
train_add1 = gen_add_1(click_df)

In [14]:
train_add1.shape

(1600185, 8)

In [15]:
df = df.merge(train_add1, on=['user_id', 'phrase', 'item_count'], how='left')

In [16]:
df.shape

(17383059, 48)

In [17]:
df.to_csv(train_path + 'new_recall/' + file_name + '_interactive.csv', index=False)